In [13]:
from bs4 import BeautifulSoup
import re
import time
import requests
import csv


def run(url):
    fw = open('reviews.csv', 'w', encoding='utf8', newline='')  # output file

    writer = csv.writer(fw, lineterminator='\n')  # create a csv writer for this file
    writer.writerow(['Critic Name', 'Review Text', 'Review Date', 'Critic Platform', 'Rating'])


    for i in range(5):  # try 5 times
        # send a request to access the url
        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})
        if response:  # explanation on response codes: https://realpython.com/python-requests/#status-codes
            break  # we got the file, break the loop
        else:
            print('fail', i)
            time.sleep(2)  # wait 2 secs

    # all five attempts failed, return None
    if not response:
        return None

    html = response.text  # read in the text from the file

    soup = BeautifulSoup(html, 'html.parser')  # parse the html

    reviews = soup.findAll('div', {'class': 'review-row'})  # get all the review divs

    for review in reviews:

        critic_name, review_text, review_date, critic_platform, rating = 'NA', 'NA', 'NA', 'NA', 'NA'  # initialize fields

        critic_chunk = review.find('a', {'href': re.compile('/critics/')})
        if critic_chunk:
            critic_name = critic_chunk.text.strip()

        platform_chunk = review.find('a', {'href': re.compile('/critics/source/')})
        if platform_chunk:
            critic_platform = platform_chunk.text.strip()

        text_chunk = review.find('p', {'class': 'review-text'})
        if text_chunk:
            review_text = text_chunk.text.strip()

        date_chunk = review.find('span', {'data-qa': 'review-date'})
        if date_chunk:
            review_date = date_chunk.text.strip()


        icon = review.find('score-icon-critic')
        if icon:
            if 'fresh' in icon.get('state'):
                rating = 'fresh'
            elif 'rotten' in icon.get('state'):
                rating = 'rotten'

        writer.writerow([critic_name, review_text, review_date, critic_platform, rating])  # write to file

    fw.close()


url = 'https://www.rottentomatoes.com/m/space_jam/reviews/'
run(url)
